In [59]:
import os
import pandas as pd
import chardet
import numpy as np
import unicodedata
import re

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Fonctions utiles</h3>
</div>

In [60]:
# --- Suppression de colonnes ---
dropped_cols = []

def delete(col):
    dropped_cols.append(col)      # Ajouter la colonne à la liste
    df.drop(col, axis=1, inplace=True)  # Supprimer la colonne du DataFrame



# --- Normalisation des accents et de la casse ---
def normalize_string(text):
    """
    Convertit une chaîne en minuscules et supprime les accents/caractères diacritiques,
    tout en protégeant les booléens (True/False) et les NaN.
    """
    
    # 1. Protection contre les NaN et None
    if pd.isna(text) or text is None:
        return text
    
    text_str = str(text)

    # 2. **PROTECTION BOOLÉENNE (NOUVEAU)**
    # Nous vérifions si la chaîne (en ignorant la casse) est 'true' ou 'false'
    if text_str.lower() in ['true', 'false']:
        # On peut soit laisser la chaîne telle quelle, soit la convertir en booléen Python natif.
        # Nous la laissons en chaîne pour le moment, mais non modifiée.
        return text
    
    # 3. Traitement standard du texte (minuscules et accents)
    
    # Convertir en minuscules (UNIQUEMENT les chaînes qui ne sont pas True/False)
    text_str_lower = text_str.lower() 

    # Décomposer les caractères (NFD)
    normalized = unicodedata.normalize('NFD', text_str_lower)
    
    # Retirer les marques d'accent
    text_no_accents = re.sub(r'[\u0300-\u036f]', '', normalized)
    
    return text_no_accents


def normalize_all_text_columns(df):
    """
    Applique la normalisation à toutes les colonnes de type 'object' ou 'string' d'un DataFrame.
    """
    string_cols = df.select_dtypes(include=['object', 'string']).columns.tolist()
    
    print(f"Normalisation des colonnes de texte : {string_cols}")

    for col in string_cols:
        # Utiliser la fonction sécurisée
        df[col] = df[col].apply(normalize_string)
        
    return df

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>Detection du format d'encodage du fichier source</p>
</div>

In [61]:
with open('houses_madrid.csv', 'rb') as file :
    encodage = chardet.detect(file.read(10000))

print(encodage)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>Import du fichier source avec le bon encodage</p>
</div>

In [62]:
pd.options.display.max_columns = None
df = pd.read_csv("houses_Madrid.csv", encoding='utf-8', index_col=1)
# df = pd.read_csv("raw_data/houses_Madrid.csv", encoding='utf_8_sig')

In [63]:
df.head(1)

,Unnamed: 0,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,raw_address,is_exact_address_hidden,street_name,street_number,portal,floor,is_floor_under,door,neighborhood_id,operation,rent_price,rent_price_by_area,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,are_pets_allowed,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_furnished,is_kitchen_equipped,is_accessible,has_green_zones,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,0,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,NaN,NaN,"Calle de Godella, 64",False,Calle de Godella,64,NaN,3,False,NaN,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,sale,471,NaN,False,85000,1328,True,HouseType 1: Pisos,False,False,1960.0,NaN,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,False,NaN,NaN,NaN,NaN,False,True,False,False


In [64]:
# Appliquer la normalisation au DataFrame après l'importation
df = normalize_all_text_columns(df)

print("\nNettoyage de l'alphabet et conversion en minuscules terminés. Vous pouvez maintenant procéder à la classification.")

Normalisation des colonnes de texte : ['title', 'subtitle', 'raw_address', 'street_name', 'street_number', 'floor', 'is_floor_under', 'neighborhood_id', 'operation', 'house_type_id', 'is_new_development', 'has_central_heating', 'has_individual_heating', 'has_ac', 'has_fitted_wardrobes', 'has_lift', 'is_exterior', 'has_garden', 'has_pool', 'has_terrace', 'has_balcony', 'has_storage_room', 'is_accessible', 'has_green_zones', 'energy_certificate', 'is_parking_included_in_price', 'is_orientation_north', 'is_orientation_west', 'is_orientation_south', 'is_orientation_east']

Nettoyage de l'alphabet et conversion en minuscules terminés. Vous pouvez maintenant procéder à la classification.


In [65]:
df.head(5)

,Unnamed: 0,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,raw_address,is_exact_address_hidden,street_name,street_number,portal,floor,is_floor_under,door,neighborhood_id,operation,rent_price,rent_price_by_area,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,are_pets_allowed,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_furnished,is_kitchen_equipped,is_accessible,has_green_zones,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,0,"piso en venta en calle de godella, 64","san cristobal, madrid",64.0,60.0,2,1.0,NaN,NaN,NaN,NaN,"calle de godella, 64",False,calle de godella,64,NaN,3,False,NaN,neighborhood 135: san cristobal (1308.89 €/m2)...,sale,471,NaN,False,85000,1328,True,housetype 1: pisos,False,False,1960.0,NaN,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d,False,NaN,NaN,NaN,NaN,False,True,False,False
21741,1,piso en venta en calle de la del manojo de rosas,"los angeles, madrid",70.0,NaN,3,1.0,NaN,NaN,NaN,NaN,calle de la del manojo de rosas,True,calle de la del manojo de rosas,NaN,NaN,4,False,NaN,neighborhood 132: los angeles (1796.68 €/m2) -...,sale,666,NaN,False,129900,1856,True,housetype 1: pisos,True,False,NaN,NaN,NaN,NaN,NaN,True,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,en tramite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21740,2,"piso en venta en calle del talco, 68","san andres, madrid",94.0,54.0,2,2.0,NaN,NaN,NaN,NaN,"calle del talco, 68",False,calle del talco,68,NaN,1,False,NaN,neighborhood 134: san andres (1617.18 €/m2) - ...,sale,722,NaN,False,144247,1535,True,housetype 1: pisos,False,False,NaN,False,True,NaN,NaN,True,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21739,3,piso en venta en calle pedro jimenez,"san andres, madrid",64.0,NaN,2,1.0,NaN,NaN,NaN,NaN,calle pedro jimenez,True,calle pedro jimenez,NaN,NaN,bajo,True,NaN,neighborhood 134: san andres (1617.18 €/m2) - ...,sale,583,NaN,False,109900,1717,True,housetype 1: pisos,False,False,1955.0,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN,True,NaN,en tramite,False,NaN,NaN,NaN,NaN,False,False,True,False
21738,4,piso en venta en carretera de villaverde a val...,"los rosales, madrid",108.0,90.0,2,2.0,NaN,NaN,NaN,NaN,carretera de villaverde a vallecas,True,carretera de villaverde a vallecas,NaN,NaN,4,False,NaN,neighborhood 133: los rosales (1827.79 €/m2) -...,sale,1094,NaN,False,260000,2407,True,housetype 1: pisos,False,False,2003.0,NaN,NaN,NaN,True,True,True,True,NaN,True,NaN,NaN,True,NaN,NaN,NaN,True,en tramite,True,NaN,NaN,True,0.0,True,True,True,True


<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>La colonne 'id' (index 1) ne contient que des valeurs unique, elle peut servir de colonne index<br>
    La colonne (index 0) n'apporte rien, on peut la supprimer</p>
</div>

In [66]:
df.head(1)

,Unnamed: 0,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,raw_address,is_exact_address_hidden,street_name,street_number,portal,floor,is_floor_under,door,neighborhood_id,operation,rent_price,rent_price_by_area,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,are_pets_allowed,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_furnished,is_kitchen_equipped,is_accessible,has_green_zones,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,0,"piso en venta en calle de godella, 64","san cristobal, madrid",64.0,60.0,2,1.0,NaN,NaN,NaN,NaN,"calle de godella, 64",False,calle de godella,64,NaN,3,False,NaN,neighborhood 135: san cristobal (1308.89 €/m2)...,sale,471,NaN,False,85000,1328,True,housetype 1: pisos,False,False,1960.0,NaN,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d,False,NaN,NaN,NaN,NaN,False,True,False,False


In [67]:
# Exemple si la colonne d'index s'appelle 'Unnamed: 0'
df = df.drop(columns=['Unnamed: 0'])

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>colonnes vides</h3>
    <p>Il faut les supprimer</p>
</div>


In [68]:
empty = df.columns[df.isna().all()].tolist()

for col in empty:
    delete(col)
dropped_cols

['latitude',
 'longitude',
 'portal',
 'door',
 'rent_price_by_area',
 'are_pets_allowed',
 'is_furnished',
 'is_kitchen_equipped',
 'has_private_parking',
 'has_public_parking']

In [69]:
df.shape

(21742, 46)

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Variables à valeurs unique, une seule modalité</h3>
    <p>A supprimer</p>
</div>

In [70]:
# Détection des variables qui n'ont qu'une seule modalité
# NaN étant considéré comme une modalité .....  (dropna=False)
unique_value_cols = df.columns[df.nunique(dropna=False)==1].tolist()
unique_value_cols

['operation', 'is_rent_price_known', 'is_buy_price_known']

In [71]:
for col in unique_value_cols:
    delete(col)
df.shape

(21742, 43)

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Variables non correllée à la target</h3>
    <p>à supprimer</p>
</div>


In [72]:
useless_cols = ['raw_address',
                'street_name',
                'street_number'
]

for col in useless_cols:
    delete(col)
df.shape

(21742, 40)

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Typage</h3>
    <p>Les colonne de float et Nan<br>A arrondir pour les convertir en 'nullable int'</p>
</div>


In [73]:
df['n_floors'].nunique()

6

In [74]:
df['n_floors'].unique()

array([nan,  2.,  3.,  4.,  1.,  5.,  7.])

In [75]:
df['n_floors'].dtype

dtype('float64')

In [76]:
# Converti en entier nullable : de 0 à 254 et NaN ou plus

# Uint8
cols_to_uint8 = ['n_floors', 'n_bathrooms']
for col in cols_to_uint8:
    df[col] = df[col].round().astype('UInt8')

# UInt16
cols_to_uint16 = ['built_year', 'sq_mt_built', 'sq_mt_useful']
for col in cols_to_uint16:
    df[col] = df[col].round().astype('UInt16')

# UInt64
cols_to_uint64 = ['parking_price', 'sq_mt_allotment']
for col in cols_to_uint64:
    df[col] = df[col].round().astype('UInt64')


In [77]:
df['n_floors'].unique()

<IntegerArray>
[<NA>, 2, 3, 4, 1, 5, 7]
Length: 7, dtype: UInt8

In [78]:
df['n_bathrooms'].unique()

<IntegerArray>
[1, 2, 3, 4, 6, <NA>, 5, 14, 7, 9, 8, 10, 13, 12, 15, 11, 16]
Length: 17, dtype: UInt8

In [79]:
df['parking_price'].unique()

<IntegerArray>
[  <NA>,      0,  10000,  15000,  20000,  14210,  11000,  25000,     20,
   5000,  12500,   7000,  12000,  17000,  14000,     35,   9000,     10,
   6000,  18000,  16000,  30000,  29000,  35000,  26000,     90,  22000,
  50000,  16950,  24264,  36000,  45000,  27000,  19000, 100000,  40000,
    120,  37000,   4150,  21000, 380000,  70000,  80000, 600000, 150000,
  75000,  60000,  19500,    100,  24000,  13000,     70,      1,  13750,
    150,  19900,  55000,    200,  85000,  65000,  69000,  48500,  90000,
  32000,  23000,     60,    180,  42000,   5403,    190,  33000,  16500,
    130,  78000,  51500,  44000,  95000, 120000,    185,  39000,    250,
     50,  14550,    165,    110]
Length: 85, dtype: UInt64

In [80]:
df.head(5)

,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,is_exact_address_hidden,floor,is_floor_under,neighborhood_id,rent_price,buy_price,buy_price_by_area,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_accessible,has_green_zones,energy_certificate,has_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,"piso en venta en calle de godella, 64","san cristobal, madrid",64,60,2,1,<NA>,<NA>,False,3,False,neighborhood 135: san cristobal (1308.89 €/m2)...,471,85000,1328,housetype 1: pisos,False,False,1960,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d,False,NaN,<NA>,False,True,False,False
21741,piso en venta en calle de la del manojo de rosas,"los angeles, madrid",70,<NA>,3,1,<NA>,<NA>,True,4,False,neighborhood 132: los angeles (1796.68 €/m2) -...,666,129900,1856,housetype 1: pisos,True,False,<NA>,NaN,NaN,NaN,True,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN,en tramite,False,NaN,<NA>,NaN,NaN,NaN,NaN
21740,"piso en venta en calle del talco, 68","san andres, madrid",94,54,2,2,<NA>,<NA>,False,1,False,neighborhood 134: san andres (1617.18 €/m2) - ...,722,144247,1535,housetype 1: pisos,False,False,<NA>,False,True,NaN,True,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN,no indicado,False,NaN,<NA>,NaN,NaN,NaN,NaN
21739,piso en venta en calle pedro jimenez,"san andres, madrid",64,<NA>,2,1,<NA>,<NA>,True,bajo,True,neighborhood 134: san andres (1617.18 €/m2) - ...,583,109900,1717,housetype 1: pisos,False,False,1955,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,True,True,NaN,en tramite,False,NaN,<NA>,False,False,True,False
21738,piso en venta en carretera de villaverde a val...,"los rosales, madrid",108,90,2,2,<NA>,<NA>,True,4,False,neighborhood 133: los rosales (1827.79 €/m2) -...,1094,260000,2407,housetype 1: pisos,False,False,2003,NaN,NaN,True,True,True,True,NaN,True,NaN,NaN,True,NaN,True,en tramite,True,True,0,True,True,True,True


<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Variables à 2 modalités : True et NaN</h3>
    <p>A remplacer par 0/1</p>
</div>

In [81]:
true_nan_cols = [ 'has_pool',
                'has_terrace', 
                'has_garden', 
                'has_balcony', 
                'has_storage_room', 
                'has_ac', 
                'has_fitted_wardrobes', 
                'is_accessible',
                'has_green_zones',
            ]
for col in true_nan_cols:
    # Tout ce qui est True devient 1, tout le reste (NaN, None, '', False, etc.) devient 0
    df[col] = df[col].apply(lambda x: 1 if x is True else 0).astype('UInt8')

In [82]:
df['has_garden'].unique()

<IntegerArray>
[0, 1]
Length: 2, dtype: UInt8

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Variables booléennes</h3>
    <p>A remplacer par 0/1</p>
</div>

In [83]:
boolean_cols = ['is_exact_address_hidden', 'has_parking', 'is_renewal_needed']

for col in boolean_cols:
    # True -> 1, False -> 0, NaN reste NaN
    df[col] = df[col].map({True: 1, False: 0}).astype('UInt8')


In [84]:
df['has_parking'].unique()

<IntegerArray>
[0, 1]
Length: 2, dtype: UInt8

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Variables booléennes avec Nan</h3>
    <p>A convertir en "nullable int", c'est à dire en `0`, `1` et `Nan`</div>

In [85]:
df.head(1)

,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,is_exact_address_hidden,floor,is_floor_under,neighborhood_id,rent_price,buy_price,buy_price_by_area,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_accessible,has_green_zones,energy_certificate,has_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,"piso en venta en calle de godella, 64","san cristobal, madrid",64,60,2,1,<NA>,<NA>,0,3,False,neighborhood 135: san cristobal (1308.89 €/m2)...,471,85000,1328,housetype 1: pisos,0,False,1960,NaN,NaN,1,0,False,True,0,0,0,0,0,0,0,d,0,NaN,<NA>,False,True,False,False


In [86]:
true_false_nan_cols = [ 'is_orientation_east', 
                'is_orientation_west', 
                'is_orientation_south', 
                'is_orientation_north',
                'has_central_heating',
                'has_individual_heating',
                'has_lift',
                'is_exterior',
                'is_parking_included_in_price',
                'is_floor_under'
            ]

for col in true_false_nan_cols :
    # Convertit en type booléen nullable
    df[col] = df[col].astype('boolean')
    # Puis map True -> 1, False -> 0, <NA> reste <NA>
    df[col] = df[col].map(lambda x: 1 if x is True else (0 if x is False else pd.NA)).astype('UInt8')



In [87]:
df['is_orientation_east'].unique()

<IntegerArray>
[0, <NA>, 1]
Length: 3, dtype: UInt8

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Energy_certificat</h3>
    <p>A convertir les srt en NaN, les chiffres en int Ordinal</div>

In [88]:
# Liste des valeurs de texte à remplacer
valeurs_a_remplacer = [
    'no indicado',      # Chaîne exacte
    'inmueble exento',  # Chaîne exacte
    '^en tr.*'          # Toutes les chaînes qui commencent par 'en tr'
]

df['energy_certificate'] = df['energy_certificate'].replace(
    valeurs_a_remplacer,
    np.nan,
    regex=True  # Active le moteur d'expressions régulières (regex)
)

# Converti les valeurs en type numérique (les NaN textuels en NaN flottant)
# L'argument errors='coerce' garantit que si une chaîne non traitée subsiste, elle devient aussi NaN.
df['energy_certificate'] = pd.to_numeric(
    df['energy_certificate'],
    errors='coerce'
)

# Le type 'UInt8' est suffisant pour la plupart des notes ou catégories de certification (0 à 255).
df['energy_certificate'] = df['energy_certificate'].astype('UInt8')

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>built_year</h3>
    <p>suprime la valeur 8170<br>ou remplace par nan

In [89]:
# # Suppression
# df_clean = df[df['built_year'] != 8170]
# df = df_clean

# remplace par nan
df['built_year'] = df['built_year'].replace(8170, np.nan)


<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>title devient product</h3>
    <p>simplifie les valeurs et renomme la colonne</p></div>

In [90]:
# dictionnaire défini avec les termes les plus spécifiques en premier : 'casa o chalet' est détecté avant 'casa'.
to_replace = {
    'piso': 'piso',
    'casa o chalet': 'casa o chalet',
    'finca': 'finca',
    'chalet adosado': 'chalet adosado',
    'chalet pareado': 'chalet pareado',
    'estudio': 'estudio',
    'atico': 'atico',
    'duplex': 'duplex',
    'casa': 'casa',
}

# 1. Définir la fonction de classification
def rename(title, mapping):
    """
    Recherche la première clé du mapping présente dans le titre
    et renvoie la valeur de remplacement correspondante.
    """
    # Convertir en chaîne de caractères et en minuscules pour une recherche robuste
    title_str = str(title).lower()

    # Itérer sur le dictionnaire dans l'ordre de priorité
    for search_key, assigned_value in mapping.items():
        # Vérifie si la clé est présente dans le titre
        if search_key in title_str:
            return assigned_value
    
    # Si aucun type de propriété n'est trouvé
    return np.nan # Vous pouvez utiliser 'Autre' ou une autre valeur par défaut


# 2. Appliquer la fonction pour créer la nouvelle colonne 'type'
df['product'] = df['title'].apply(lambda x: rename(x, to_replace))

print("La colonne 'type' a été créée en classifiant les valeurs de 'title'.")

La colonne 'type' a été créée en classifiant les valeurs de 'title'.


In [91]:
df['product'].isna().sum()

np.int64(0)

In [92]:
df['house_type_id'].unique()

array(['housetype 1: pisos', 'housetype 4: duplex', nan,
       'housetype 5: aticos', 'housetype 2: casa o chalet'], dtype=object)

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>eighborhood_id</h3>
    <p>Ne garder que le numéro du quartier</p></div>

In [93]:
df['neighborhood_id'].values

array(['neighborhood 135: san cristobal (1308.89 €/m2) - district 21: villaverde',
       'neighborhood 132: los angeles (1796.68 €/m2) - district 21: villaverde',
       'neighborhood 134: san andres (1617.18 €/m2) - district 21: villaverde',
       ...,
       'neighborhood 9: campo de las naciones-corralejos (3417.44 €/m2) - district 2: barajas',
       'neighborhood 9: campo de las naciones-corralejos (3417.44 €/m2) - district 2: barajas',
       'neighborhood 3: imperial (4098.59 €/m2) - district 1: arganzuela'],
      shape=(21742,), dtype=object)

In [94]:
df['neighborhood'] = df['neighborhood_id'].str.extract(r'neighborhood (\d+)')
df['neighborhood'].values


array(['135', '132', '134', ..., '9', '9', '3'],
      shape=(21742,), dtype=object)

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Suppression de colonnes</h3>
    <ul><li>title</li><li>subtitle</li><li>house_type_id</li><li>is_exact_address_hidden</li><li>neighborhood_id</li></ul></div>

In [95]:
to_drop =['title', 'subtitle', 'house_type_id', 'is_exact_address_hidden', 'neighborhood_id']
delete(to_drop)

In [96]:
df.head(1)

,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,floor,is_floor_under,rent_price,buy_price,buy_price_by_area,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_accessible,has_green_zones,energy_certificate,has_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east,product,neighborhood
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,64,60,2,1,<NA>,<NA>,3,0,471,85000,1328,0,False,1960,<NA>,<NA>,1,0,0,1,0,0,0,0,0,0,0,<NA>,0,<NA>,<NA>,0,1,0,0,piso,135


In [97]:
df.shape

(21742, 37)

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h2>Export en .feather</h2>
</div>

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h2>Export en .csv</h2>
</div>

In [102]:
df.to_csv("houses_madrid_cleaned.csv", index=False, encoding="utf-8")


In [ ]:
df.head(1)

,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,floor,is_floor_under,rent_price,buy_price,buy_price_by_area,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_accessible,has_green_zones,energy_certificate,has_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east,product,neighborhood
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,64,60,2,1,<NA>,<NA>,3,0,471,85000,1328,0,False,1960,<NA>,<NA>,1,0,0,1,0,0,0,0,0,0,0,<NA>,0,<NA>,<NA>,0,1,0,0,piso,135


In [ ]:
df.shape

(21742, 37)